In [16]:
#!mkdir ./tmp
!pip install magenta --user --build ./tmp

  Using cached https://files.pythonhosted.org/packages/df/b9/a2432f924cd042f2847164b2cf02ab19dd86b9086cf681e336371d369782/magenta-0.3.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/49/25/1a8b1290b51fb0d4a499c3285b635c005e30b8ff423fb116db61f3d80ca5/python-rtmidi-1.1.0.zip
  Running setup.py bdist_wheel for python-rtmidi ... done
  Stored in directory: /home/jheuristic/.cache/pip/wheels/42/ac/9e/d5c633fcc35e0f31e4eebc3e7a5ea5383bc835f1f20c901a81
Successfully built python-rtmidi
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
!pip install --user tensorflow==1.4.0 --build ./tmp

  Using cached https://files.pythonhosted.org/packages/86/9f/be0165c6eefd841e6928e54d3d083fa174f92d640fdc52f73a33dc9c54d1/tensorflow-1.4.0-cp36-cp36m-manylinux1_x86_64.whl
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import sys
sys.path.insert(0, "/home/jheuristic/.local/lib/python3.6/")

In [11]:
import tensorflow as tf

/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [20]:
CHECKPOINT_URL = 'http://download.magenta.tensorflow.org/models/onsets_frames_transcription/checkpoint.zip'
!wget -O checkpoint.zip {CHECKPOINT_URL}
!unzip -o checkpoint.zip
CHECKPOINT_DIR = './train'

wget: /opt/conda/anaconda3/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/anaconda3/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/anaconda3/lib/libssl.so.1.0.0: no version information available (required by wget)
--2018-05-19 14:41:10--  http://download.magenta.tensorflow.org/models/onsets_frames_transcription/checkpoint.zip
Resolving download.magenta.tensorflow.org (download.magenta.tensorflow.org)... 172.217.9.16, 2607:f8b0:4000:812::2010
Connecting to download.magenta.tensorflow.org (download.magenta.tensorflow.org)|172.217.9.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53849639 (51M) [application/zip]
Saving to: ‘checkpoint.zip’

checkpoint.zip      100%[===================>]  51.35M  79.4MB/s    in 0.6s    

2018-05-19 14:41:11 (79.4 MB/s) - ‘checkpoint.zip’ saved [53849639/53849639]

Archive:  checkpoint.zip
   creating: train/
  inflating: train/model.ckpt-500

In [21]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import librosa
import numpy as np

#from google.colab import files

from magenta.common import tf_utils
from magenta.music import audio_io
import magenta.music as mm
from magenta.models.onsets_frames_transcription import model
from magenta.models.onsets_frames_transcription import constants
from magenta.models.onsets_frames_transcription import data
from magenta.models.onsets_frames_transcription import infer_util
from magenta.music import midi_io
from magenta.protobuf import music_pb2

## Define model and load checkpoint
## Only needs to be run once.

acoustic_checkpoint = tf.train.latest_checkpoint(CHECKPOINT_DIR)
print('acoustic_checkpoint=' + acoustic_checkpoint)
hparams =  tf_utils.merge_hparams(
      constants.DEFAULT_HPARAMS, model.get_default_hparams())

with tf.Graph().as_default():
  examples = tf.placeholder(tf.string, [None])

  num_dims = constants.MIDI_PITCHES

  batch, iterator = data.provide_batch(
      batch_size=1,
      examples=examples,
      hparams=hparams,
      is_training=False,
      truncated_length=0)

  model.get_model(batch, hparams, is_training=False)

  session = tf.Session()
  saver = tf.train.Saver()
  saver.restore(session, acoustic_checkpoint)

  onset_probs_flat = tf.get_default_graph().get_tensor_by_name(
      'onsets/onset_probs_flat:0')
  frame_probs_flat = tf.get_default_graph().get_tensor_by_name(
     'frame_probs_flat:0')

acoustic_checkpoint=./train/model.ckpt-50002
INFO:tensorflow:Restoring parameters from ./train/model.ckpt-50002


In [22]:
uploaded ='/home/jheuristic/data/wav/alwaysforever.mid.wav'

to_process = []
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  open(fn, 'w').write(uploaded[fn])
  wav_data = audio_io.samples_to_wav_data(
      librosa.util.normalize(librosa.core.load(fn, sr=hparams.sample_rate)[0]),
      hparams.sample_rate)

  example = tf.train.Example(features=tf.train.Features(feature={
      'id':
          tf.train.Feature(bytes_list=tf.train.BytesList(
              value=[fn.encode('utf-8')]
          )),
      'sequence':
          tf.train.Feature(bytes_list=tf.train.BytesList(
              value=[music_pb2.NoteSequence().SerializeToString()]
          )),
      'audio':
          tf.train.Feature(bytes_list=tf.train.BytesList(
              value=[wav_data]
          )),
  }))
  to_process.append(example.SerializeToString())
  print('Processing complete for', fn)
  
  
session.run(iterator.initializer, {examples: to_process})

AttributeError: 'str' object has no attribute 'keys'

In [5]:
!git clone https://github.com/tensorflow/magenta

Cloning into 'magenta'...
remote: Counting objects: 7320, done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 7320 (delta 9), reused 8 (delta 2), pack-reused 7288
Receiving objects: 100% (7320/7320), 26.74 MiB | 16.97 MiB/s, done.
Resolving deltas: 100% (4904/4904), done.
Checking connectivity... done.


In [12]:
import sys
sys.path.append("/mnt/jheuristic/Google_magenta/magenta_repo/")

In [13]:
import magenta

ImportError: cannot import name 'music_pb2'